In [1]:
pip install langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 28.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.5 which is incompatible.
google-generativeai 0.8.5 requires google-ai-

In [2]:
import base64
import getpass
import json
import os

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

In [3]:
os.environ['GOOGLE_API_KEY'] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

In [25]:
PROMPT = """
You are given an image of a receipt. Please read the content into JSON format:

```
{
  "menus": [
    {
      "name": <item_name>,
      "count": <purchased_count>,
      "price": <total_price>
    },
    "subtotal": <total_price_without_tax>
    "tax": <total_tax>
    "service_charge": <service_value>
    "others": <other_additional_fess_if_any>
  ],
  "total": <total_price_in_receipt>
}
```

return only in JSON format
"""

IMAGE_PATH = "receipt_darikopi.jpg"

with open(IMAGE_PATH, 'rb') as f:
  image_bytes = f.read()
encoded = base64.b64encode(image_bytes).decode("utf-8")
data_uri = f'data:image/jpeg;base64, {encoded}'

message = HumanMessage(
    content = [
        {'type': 'text', 'text': PROMPT},
        {'type': 'image_url', 'image_url': data_uri},
    ]
)
response = llm.invoke([message])
print(response.content)

```json
{
  "menus": [
    {
      "name": "CHEESE CAKE",
      "count": 1,
      "price": 45000
    },
    {
      "name": "MINERAL WATER",
      "count": 1,
      "price": 18000
    },
    {
      "name": "SAUSAGE SMOKED BF PZ",
      "count": 1,
      "price": 85000
    },
    {
      "name": "SPAGHETTI FANTASIA",
      "count": 1,
      "price": 55000
    },
    {
      "name": "SUMMER ORANGE NOIR",
      "count": 1,
      "price": 35000
    },
    {
      "name": "TIRAMISU COFFEE",
      "count": 2,
      "price": 70000
    },
    {
      "name": "TIRAMISU MATCHA",
      "count": 1,
      "price": 38000
    }
  ],
  "subtotal": 346000,
  "tax": 34600,
  "service_charge": 17300,
  "others": null,
  "total": 397900
}
```


In [26]:
json_string = response.content.replace("```json", "").replace("```", "")
receipt_dict = json.loads(json_string)
receipt_dict


{'menus': [{'name': 'CHEESE CAKE', 'count': 1, 'price': 45000},
  {'name': 'MINERAL WATER', 'count': 1, 'price': 18000},
  {'name': 'SAUSAGE SMOKED BF PZ', 'count': 1, 'price': 85000},
  {'name': 'SPAGHETTI FANTASIA', 'count': 1, 'price': 55000},
  {'name': 'SUMMER ORANGE NOIR', 'count': 1, 'price': 35000},
  {'name': 'TIRAMISU COFFEE', 'count': 2, 'price': 70000},
  {'name': 'TIRAMISU MATCHA', 'count': 1, 'price': 38000}],
 'subtotal': 346000,
 'tax': 34600,
 'service_charge': 17300,
 'others': None,
 'total': 397900}

In [20]:
menu_feby = ['CHEESE CAKE', 'MINERAL WATER']
menu_tona = ['SAUSAGE SMOKED BF PZ', 'TIRAMISU COFFEE']
menu_ida  = ['SPAGHETTI FANTASIA', 'TIRAMISU MATCHA']
menu_ika  = ['SUMMER ORANGE NOIR', 'TIRAMISU COFFEE']

total_feby = 0
total_tona = 0
total_ida = 0
total_ika = 0

list_nama_menu = [m['name'] for m in receipt_dict['menus']]
list_harga_menu = [m['price'] for m in receipt_dict['menus']]
list_qty_menu = [m['count'] for m in receipt_dict['menus']]

for nama_menu, harga_menu, qty in zip(list_nama_menu, list_harga_menu, list_qty_menu):
  quantity = qty if qty else 1

  harga_per_item = harga_menu / quantity
  harga_per_item = int(harga_per_item)

  if nama_menu in menu_feby:
      total_feby += harga_per_item
  if nama_menu in menu_tona:
      total_tona += harga_per_item
  if nama_menu in menu_ida:
      total_ida += harga_per_item
  if nama_menu in menu_ika:
      total_ika += harga_per_item

In [24]:
total_ika

70000